# CNN Model with Cifar10 Dataset ,batch-size=64.

# Libraries, Dataset

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import random_split
transform_Training = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),transforms.RandomHorizontalFlip(),transforms.RandomCrop(size=32)
])
transform_Testing = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_Training)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_Testing)

Files already downloaded and verified
Files already downloaded and verified


In [2]:
print(len(train_dataset))
print(len(test_dataset))

50000
10000


## Dataloaders

In [3]:
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)
classes=( 'plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship','truck')

## Preparing the Model

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.conv5 = nn.Conv2d(256, 512, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        
        self.fc1 = nn.Linear(512 * 2 * 2, 1024)  
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))
        x = self.pool(F.relu(self.conv5(x)))
        
        x = x.view(-1, 512 * 2 * 2)  
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


model = CNN()
inputs = torch.randn(10, 3, 32, 32)  
outputs = model(inputs)  
print(outputs.size())  


torch.Size([10, 10])


## Loss,Optimizier

In [5]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.005)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

CNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=2048, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=10, bias=True)
)

## Training

In [6]:
num_epochs = 50
for epoch in range(num_epochs):
    model.train()  
    running_loss = 0.0

    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    
    print(f"Epoch {epoch + 1}, Loss: {running_loss / len(trainloader)}")

print('Finished Training')


Epoch 1, Loss: 2.3028199901361295
Epoch 2, Loss: 2.3024268299722306
Epoch 3, Loss: 2.302167018661109
Epoch 4, Loss: 2.3019025712976675
Epoch 5, Loss: 2.3015521487311634
Epoch 6, Loss: 2.3009799788979923
Epoch 7, Loss: 2.299899714072342
Epoch 8, Loss: 2.2973864127303023
Epoch 9, Loss: 2.2886937048733995
Epoch 10, Loss: 2.233994800111522
Epoch 11, Loss: 2.0978988023365246
Epoch 12, Loss: 1.9938219765872907
Epoch 13, Loss: 1.942778722099636
Epoch 14, Loss: 1.881997673712728
Epoch 15, Loss: 1.812597309841829
Epoch 16, Loss: 1.731901148396075
Epoch 17, Loss: 1.6617470958348735
Epoch 18, Loss: 1.6023522132193035
Epoch 19, Loss: 1.547711819791428
Epoch 20, Loss: 1.5033811913121997
Epoch 21, Loss: 1.4625167456429329
Epoch 22, Loss: 1.4246573759161907
Epoch 23, Loss: 1.3865335083678556
Epoch 24, Loss: 1.355269330922905
Epoch 25, Loss: 1.3229137305408487
Epoch 26, Loss: 1.2883659415995068
Epoch 27, Loss: 1.2539501298419045
Epoch 28, Loss: 1.221839338159927
Epoch 29, Loss: 1.1832545718268666
Epoc

## Evaluation

In [7]:
model.eval()
correct = 0
total = 0
predicted_labels = []
true_labels = []
with torch.no_grad():
    for inputs, labels in testloader:
        inputs, labels = inputs.to(device), labels.to(device)
        # Forward pass
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        predicted_labels.extend(predicted.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())
# accuracy
accuracy = correct / total
# F-score
from sklearn.metrics import f1_score
f_score = f1_score(true_labels, predicted_labels, average='weighted')
print(f"Accuracy: {accuracy}")
print(f"F-score: {f_score}")

Accuracy: 0.6843
F-score: 0.6839829613388776
